In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Twitter Stock Market Dataset.csv to Twitter Stock Market Dataset.csv


In [8]:
data = "C:\\Users\\Hp\\Downloads\\ML\\archive (3)\\Twitter Stock Market Dataset.csv"
data = pd.read_csv("Twitter Stock Market Dataset.csv")


In [18]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-11-07,45.099998,50.090000,44.000000,44.900002,44.900002,117701670.0
1,2013-11-08,45.930000,46.939999,40.685001,41.650002,41.650002,27925307.0
2,2013-11-11,40.500000,43.000000,39.400002,42.900002,42.900002,16113941.0
3,2013-11-12,43.660000,43.779999,41.830002,41.900002,41.900002,6316755.0
4,2013-11-13,41.029999,42.869999,40.759998,42.599998,42.599998,8688325.0
...,...,...,...,...,...,...,...
2259,2022-10-28,NaN,NaN,NaN,NaN,NaN,NaN
2260,2022-10-31,NaN,NaN,NaN,NaN,NaN,NaN
2261,2022-11-01,NaN,NaN,NaN,NaN,NaN,NaN
2262,2022-11-02,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
data.dropna()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-11-07,45.099998,50.090000,44.000000,44.900002,44.900002,117701670.0
1,2013-11-08,45.930000,46.939999,40.685001,41.650002,41.650002,27925307.0
2,2013-11-11,40.500000,43.000000,39.400002,42.900002,42.900002,16113941.0
3,2013-11-12,43.660000,43.779999,41.830002,41.900002,41.900002,6316755.0
4,2013-11-13,41.029999,42.869999,40.759998,42.599998,42.599998,8688325.0
...,...,...,...,...,...,...,...
2254,2022-10-21,50.000000,50.750000,49.549999,49.889999,49.889999,51209029.0
2255,2022-10-24,50.709999,51.860001,50.520000,51.520000,51.520000,22987553.0
2256,2022-10-25,52.415001,53.180000,52.200001,52.779999,52.779999,35077848.0
2257,2022-10-26,52.950001,53.500000,52.770000,53.349998,53.349998,28064973.0


In [10]:
data['Date'] = pd.to_datetime(data['Date'])

In [11]:
data = data.sort_values(by='Date', ascending=True)

In [12]:
scaler = MinMaxScaler(feature_range=(0, 1))
data[['Open', 'High', 'Low', 'Close', 'Volume']] = scaler.fit_transform(data[['Open', 'High', 'Low', 'Close', 'Volume']])

In [13]:
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data.iloc[i:i+n_steps, 1:].values)  # Input features (excluding 'Date' column)
        y.append(data.iloc[i+n_steps, 4])            # Target variable ('Close' price)
    return np.array(X), np.array(y)

In [14]:
n_steps = 60

# Create sequences of input-output pairs
X, y = create_sequences(data, n_steps)

# Split the dataset into training and testing sets
split_ratio = 0.8
split_index = int(split_ratio * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Print the shapes of training and testing sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (1763, 60, 6)
X_test shape: (441, 60, 6)
y_train shape: (1763,)
y_test shape: (441,)


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define the LSTM model
model = Sequential([
    LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1, validation_split=0.2)

Epoch 1/10
45/45 [==============================] - 6s 76ms/step - loss: 0.0207 - val_loss: 0.0019
Epoch 2/10
45/45 [==============================] - 2s 35ms/step - loss: 0.0016 - val_loss: 0.0012
Epoch 3/10
45/45 [==============================] - 1s 29ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 4/10
45/45 [==============================] - 1s 27ms/step - loss: 8.1520e-04 - val_loss: 9.8500e-04
Epoch 5/10
45/45 [==============================] - 1s 27ms/step - loss: 6.7606e-04 - val_loss: 6.5262e-04
Epoch 6/10
45/45 [==============================] - 1s 27ms/step - loss: 5.7088e-04 - val_loss: 5.9249e-04
Epoch 7/10
45/45 [==============================] - 1s 28ms/step - loss: 4.8880e-04 - val_loss: 5.9865e-04
Epoch 8/10
45/45 [==============================] - 1s 27ms/step - loss: 4.6182e-04 - val_loss: 5.8016e-04
Epoch 9/10
45/45 [==============================] - 1s 30ms/step - loss: 4.1477e-04 - val_loss: 5.6621e-04
Epoch 10/10
45/45 [==============================] - 2s 40ms/

In [16]:
# Evaluate the model on the testing data
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)


14/14 [==============================] - 0s 10ms/step - loss: nan
Test Loss: nan
